In [1]:
using Revise, LazySets, Test, BenchmarkTools

In [2]:
h = rand(Hyperrectangle)

p = convert(HPolygon, h)

#p = HPolygon([HalfSpace(Vector(c.a), c.b) for c in constraints_list(convert(HPolygon, h))])

q = rand(HPolygon);

In [3]:
intersection(p, q)

EmptySet{Float64}(2)

In [4]:
@inferred intersection(p, q)

ErrorException: return type EmptySet{Float64} does not match inferred return type Union{EmptySet{Float64}, HPolygon}

In [7]:
@inferred intersection2(p, q)

ErrorException: return type HPolygon{Float64,Array{Float64,1}} does not match inferred return type Union{HPolygon{Float64,LazySets.Arrays.SingleEntryVector{Float64}}, HPolygon{Float64,Array{Float64,1}}}

In [23]:
function qq(z; prune=true)
    if prune
        remove_redundant_constraints!(z)
        if isempty(z)
            return EmptySet(2)
        end
    end
    return z
end

function ff(p, q; prune=true)
    z = intersection2(p, q)
    qq(z, prune=prune)
end

ff (generic function with 1 method)

In [24]:
@inferred ff(p, q)

ErrorException: return type EmptySet{Float64} does not match inferred return type Union{EmptySet{Float64}, HPolygon}

In [ ]:
@btime intersection($p, $q)

In [ ]:
ppoly = HPolytope(p.constraints)
qpoly = HPolytope(p.constraints)
@btime intersection($ppoly, $qpoly)

In [ ]:
@btime area(intersection2($p, $q))

In [ ]:
@inferred intersection2(p, q)

In [6]:
# all constraints of one polygon are processed; now add the other polygon's
# constraints
@inline function add_remaining_constraints!(c, i, c1, i1, duplicates)
    c[i+1:length(c)-duplicates] = c1[i1:length(c1)]
    return true
end

# choose the constraint of c1; the directions are different
@inline function choose_first_diff_dir!(c, i, i1, i2, c1, c2, duplicates)
    c[i] = c1[i1]
    if i1 == length(c1)
        return add_remaining_constraints!(c, i, c2, i2, duplicates)
    end
    return false
end

# choose the constraint of c1; the directions are equivalent (i.e., linearly
# dependent)
@inline function choose_first_same_dir!(c, i, i1, i2, c1, c2, duplicates)
    c[i] = c1[i1]
    if i1 == length(c1)
        if i2 < length(c2)
            return add_remaining_constraints!(c, i, c2, i2+1, duplicates)
        end
        return true
    elseif i2 == length(c2)
        return add_remaining_constraints!(c, i, c1, i1+1, duplicates)
    end
    return false
end

function intersection2(P1::HPolygon{N, VN}, P2::HPolygon{N, UN}) where {N<:Real, VN, UN}
    
    c1 = constraints_list(P1)
    c2 = constraints_list(P2)
    
    
    if isempty(c1)
       return P2
    elseif isempty(c2)
        return P1
    end
    
    
    # TODO: use common vector type of P1 and P2, see #2046
    c = Vector{LinearConstraint{N, Vector{N}}}(undef, length(c1) + length(c2))
    i1 = 1
    i2 = 1
    duplicates = 0
    for i in 1:length(c)
        if c1[i1].a <= c2[i2].a
            if c2[i2].a <= c1[i1].a
                duplicates += 1
                # constraints have the same normal vector: take the tighter one
                if is_tighter_same_dir_2D(c1[i1], c2[i2])
                    # first constraint is tighter
                    if choose_first_same_dir!(c, i, i1, i2, c1, c2, duplicates)
                        break
                    end
                else
                    # second constraint is tighter
                    if choose_first_same_dir!(c, i, i2, i1, c2, c1, duplicates)
                        break
                    end
                end
                i1 += 1
                i2 += 1
            else
                # first constraint comes first
                if choose_first_diff_dir!(c, i, i1, i2, c1, c2, duplicates)
                    break
                end
                i1 += 1
            end
        else
            # second constraint comes first
            if choose_first_diff_dir!(c, i, i2, i1, c2, c1, duplicates)
                break
            end
            i2 += 1
        end
    end
    if duplicates > 0
        deleteat!(c, length(c)-duplicates+1:length(c))
    end

    P = HPolygon(c, sort_constraints=false)
    #return P
    #if prune
        #remove_redundant_constraints!(P)
    #    return EmptySet{N}(2)
        
     #   if isempty(P)
      #      return EmptySet{N}(2)
      #  end
    #end
    return P
end

intersection2 (generic function with 1 method)

In [ ]:
@which remove_redundant_constraints!(p)

In [ ]:
z = HPolygon(constraints_list(rand(2, 2) * rand(HPolygon)), sort_constraints = true, prune = false, check_boundedness = false)

In [ ]:
@inferred intersection(p, q)

In [ ]:
@inferred area(intersection(p, q))

In [ ]:
function intersection_area(X::AbstractHyperrectangle{N},
    Y::LinearMap{N,<:AbstractHyperrectangle{N}}) where {N}

    #X_clist = X |> constraints_list
    #X_poly = HPolygon(X_clist, sort_constraints = true, prune = false, check_boundedness = false)
    X_poly = HPolygon([HalfSpace(Vector(c.a), c.b) for c in constraints_list(convert(HPolygon, X))])
    
    Y_clist = linear_map(matrix(Y), set(Y)) |> constraints_list
    Y_poly = HPolygon(Y_clist, sort_constraints = true, prune = false, check_boundedness = false)

    #println(typeof(X_poly))
    #println(typeof(Y_poly))
    
    intersection(X_poly, Y_poly) |> area
#    return _intersection_area(X_poly, Y_poly)
end

_intersection_area(p, q) = area(intersection(p, q))

In [ ]:
X = rand(Hyperrectangle)
Y = rand(2, 2) * rand(Hyperrectangle);

In [ ]:
@btime intersection_area($X, $Y)

In [ ]:
@btime convert(HPolygon, $X)

In [ ]:
@btime HPolygon(constraints_list($X), sort_constraints = true, prune = false, check_boundedness = false)

In [ ]:
LazySets.vertices_list(::EmptySet{N}) where {N} = N[]

In [ ]:
LazySets.area(::EmptySet{N}) where {N} = zero(N)

In [ ]:
@btime intersection_area($H, $X)

In [ ]:
function f(p, q)
    return area(intersection(p, q))
end

In [ ]:
@inferred f(p, q)